In [2]:
from __future__ import print_function

import json
import requests

In [3]:
server = 'http://localhost:1234/v1/'

In [4]:
def do_request(service, a1=None, a2=None, a3=None, **kwargs):
    server = 'http://localhost:1234/v1/'
    params = ''
    for a in [a1, a2, a3]:
        if a is not None:
            params += '/' + a
    req = requests.get('%s/%s%s' % (server, service, params),
                       params=kwargs,
                       headers={'Content-Type': 'application/json'})
    if not req.ok:
        req.raise_for_status()
    return req.json()

In [5]:
http://www.biomart.org/biomart/martservice?type=datasets&mart=uniprot')
rec =requests.post('networks?source=url&collection=Test')

SyntaxError: invalid syntax (<ipython-input-5-442cec344e8c>, line 1)

In [15]:
def create_from_list(network_list):
    req = requests.post(server + 'networks?source=url&collection=test', data=json.dumps(network_list),
                               headers={'Content-Type': 'application/json'})
    if not req.ok:
        req.raise_for_status()
    return json.loads(req.content)

nl = ['http://www.uniprot.org/uniprot/P04637.xml?include=yes']
#nl = ['http://rest.kegg.jp/get/hsa05200/kgml', 'http://rest.kegg.jp/get/hsa03430/kgml']
#nl = ['http://www.uniprot.org/uniprot/?query=organism:9606+AND+p53&format=xml&limit=10&sort=score']
bla = create_from_list(nl)

In [16]:
bla

[{u'networkSUID': [2000],
  u'source': u'http://www.uniprot.org/uniprot/P04637.xml?include=yes'}]

In [22]:
queries = [ 'http://www.ebi.ac.uk/Tools/webservices/psicquic/uniprot/webservices/current/search/interactor/tp53?format=tab27']
print(json.dumps(create_from_list(queries), indent=4))

[
    {
        "source": "http://www.ebi.ac.uk/Tools/webservices/psicquic/uniprot/webservices/current/search/interactor/tp53?format=tab27", 
        "networkSUID": [
            2648
        ]
    }
]
